In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.tokenize.casual import casual_tokenize
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
import joblib

[nltk_data] Downloading package punkt to /home/damian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('./data/originals/train.csv')

In [3]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [4]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def custom_tokenize(text, stemming=True, lemmatizing=True):
        tokens = casual_tokenize(text)
        if stemming:
            tokens = [stemmer.stem(token) for token in tokens]
        if lemmatizing:
            tokens = [lemmatizer.lemmatize(token) for token in tokens]
        tokens = [token.lower() for token in tokens]
        return tokens

In [7]:
tokenized_questions = [custom_tokenize(str(question).lower()) for question in df['question1'] + df['question2']]
vector_size = 100
window_size = 5

model = Word2Vec(sentences=tokenized_questions, vector_size=vector_size, window=window_size, min_count=1, workers=4)

In [8]:
def pad_vector(vector, target_length):
    if not vector:
        return [[0] * vector_size] * target_length
    elif len(vector) < target_length:
        return vector + [[0] * len(vector[0])] * (target_length - len(vector))
    elif len(vector) > target_length:
        return vector[:target_length]
    else:
        return vector

def question_to_vector(question, model, max_length):
    tokens = word_tokenize(str(question).lower())
    vector = [model.wv[word] for word in tokens if word in model.wv]
    padded_vector = pad_vector(vector, target_length=max_length)
    return padded_vector

MAX_LENGTH = 10
df['question1'] = df['question1'].apply(lambda x: question_to_vector(x, model, MAX_LENGTH))
df['question2'] = df['question2'].apply(lambda x: question_to_vector(x, model, MAX_LENGTH))
df = df[['question1', 'question2', 'is_duplicate']]

In [9]:
df

,question1,question2,is_duplicate
0,"[[-1.5032766, 0.86574936, 0.36891624, 1.712822...","[[-1.5032766, 0.86574936, 0.36891624, 1.712822...",0
1,"[[-1.5032766, 0.86574936, 0.36891624, 1.712822...","[[-1.5032766, 0.86574936, 0.36891624, 1.712822...",0
2,"[[-2.0059712, -2.4301207, -0.011298958, 1.0741...","[[-2.0059712, -2.4301207, -0.011298958, 1.0741...",0
3,"[[-3.4872358, 0.53379977, 0.6909281, -2.180745...","[[-2.1070597, 1.9086506, 2.1574728, 3.1451943,...",0
4,"[[-2.3184087, 1.7391995, -0.88098085, 0.757990...","[[-2.3184087, 1.7391995, -0.88098085, 0.757990...",0
...,...,...,...
404285,"[[-2.0059712, -2.4301207, -0.011298958, 1.0741...","[[-2.0059712, -2.4301207, -0.011298958, 1.0741...",0
404286,"[[0.52658993, -1.1519116, -1.4788607, -1.40196...","[[0.9564838, 1.2072484, 0.6437706, -0.8572106,...",1
404287,"[[-1.5032766, 0.86574936, 0.36891624, 1.712822...","[[-1.5032766, 0.86574936, 0.36891624, 1.712822...",0
404288,"[[-1.5032766, 0.86574936, 0.36891624, 1.712822...","[[0.85345, -1.2068267, 0.330036, -0.9635719, -...",0


In [10]:
del model
del tokenized_questions

In [11]:
joblib.dump(df, './data/word2vec_10_100.pkl')

['./data/word2vec_10_100.pkl']